 Poudel et al. (2024) [11] implemented feature engineering and one-hot encoding for categorical variables.
 Poudel et al. (2024) [11] explored various models such as SVM, Logistic Regression, Random Forest, Gradient Boosting Machine (GBM), and Neural Networks, demonstrating that ensemble models like Random Forest and GBM often outperform traditional classifiers.
 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import statsmodels.api as sm
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
import scipy.stats as stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
#from imblearn.pipeline import Pipeline  # Use imblearn's Pipeline to support SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn.pipeline import Pipeline

In [2]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Load your dataset
data_no_total = pd.read_csv("cleaned_data_telecom.csv")
features = data_no_total.columns.drop('churn')
target = 'churn'

# Split data
X_train, X_temp, y_train, y_temp = train_test_split(data_no_total[features], data_no_total[target], test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
numerical_features = X_train.select_dtypes(include=[np.number]).columns.tolist()

# Define a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ]
)

# Fit the preprocessor on the training data only
preprocessor.fit(X_train)

# Transform the training, validation, and test sets using the same preprocessor

X_train_preprocessed = preprocessor.transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)
X_test_preprocessed = preprocessor.transform(X_test)

# Output the shapes of the processed datasets to confirm transformation

#print("Training set shape:", X_train_preprocessed.shape)
#print("Validation set shape:", X_val_preprocessed.shape)
#print("Test set shape:", X_test_preprocessed.shape)
# Initialize MLflow experiment
mlflow.set_experiment("Churn Prediction Experiment")

# Define models to experiment with
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='linear', probability=True, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

# Loop over models
for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        
        # Log model name
        mlflow.log_param("model_name", model_name)
        
        # Train model
        model.fit(X_train, y_train)
        
        # Predict on validation set
        y_val_pred = model.predict(X_val)
        
        # Calculate metrics
        accuracy = accuracy_score(y_val, y_val_pred)
        report = classification_report(y_val, y_val_pred, output_dict=True)
        precision_yes = report['1']['precision']
        recall_yes = report['1']['recall']
        
        # Log metrics
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision_yes", precision_yes)
        mlflow.log_metric("recall_yes", recall_yes)
        
        # Log model
        mlflow.sklearn.log_model(model, model_name)
        
        # Print metrics for verification
        print(f"{model_name} Accuracy: {accuracy:.4f}")
        print(classification_report(y_val, y_val_pred))

# After each run, go to the MLflow UI to view and compare metrics


2024/10/30 17:42:18 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



ValueError: could not convert string to float: 'Female'